In [1]:
source("sources/ggrare.R") #github library: https://rdrr.io/github/gauravsk/ranacapa/
source("sources/miseqR.R")
source("sources/qiime2R.R") #github https://github.com/jbisanz/qiime2R
source('sources/abundances.R') #github https://github.com/microbiome/microbiome

library(biomformat)
library(ape)
library(cowplot)
library(cluster)
library(data.table)
library(plyr)

library(dplyr)
#library("expss")
#library(ggplot2)
#library("ggpubr")
#library(Hmisc)
#library(otuSummary)
library(phyloseq)
#library(scales)
#library(tibble)
#library(vegan)
library(yaml)

Warning message:
"package 'ape' was built under R version 3.5.3"
Warning message:
"package 'cowplot' was built under R version 3.5.1"
Loading required package: ggplot2

Warning message:
"package 'ggplot2' was built under R version 3.5.1"

Attaching package: 'cowplot'


The following object is masked from 'package:ggplot2':

    ggsave


Warning message:
"package 'data.table' was built under R version 3.5.1"
Warning message:
"package 'dplyr' was built under R version 3.5.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:plyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'phyloseq' was built under R version 3.5.1"
Warning message:
"package

In [20]:
#read in config
configfile = read_yaml('config.yaml')
data_dir=configfile$raw_data_dir
analysis=configfile$analysis_dir
output_dir=configfile$output_dir
manifest_file=configfile$clean_manifest
variable_file=configfile$variable_manifest
binf_file=configfile$binf_manifest
ref_dir=configfile$ref_dir
qza_dir=configfile$qza_dir

In [3]:
#decide which files to include, add Y or N flag
benchmarkdf = read.csv(binf_file,sep="\t")
benchmarkdf = subset(benchmarkdf,include=="Y")
rownames(benchmarkdf) = benchmarkdf$file.name
benchmarkdf

,tax,group,group.ref,tax.ref,file.name,include,classifier
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
blast_cl_gg_gg.qza,blast,closed,greengenes,greengenes,blast_cl_gg_gg.qza,Y,refdb_tax_gg.qza
blast_dada_gg.qza,blast,dada2,greengenes,greengenes,blast_dada_gg.qza,Y,refdb_tax_gg.qza
blast_dn_gg.qza,blast,denovo,NA,greengenes,blast_dn_gg.qza,Y,refdb_tax_gg.qza
blast_op_gg_gg.qza,blast,open,greengenes,greengenes,blast_op_gg_gg.qza,Y,refdb_tax_gg.qza
scikit_clgg_gg.qza,scikit,closed,greengenes,greengenes,scikit_clgg_gg.qza,Y,classify-sklearn_gg-13-8-99-nb-classifier.qza
scikit_dada_gg.qza,scikit,dada2,greengenes,greengenes,scikit_dada_gg.qza,Y,classify-sklearn_gg-13-8-99-nb-classifier.qza
scikit_dn_gg.qza,scikit,denovo,NA,greengenes,scikit_dn_gg.qza,Y,classify-sklearn_gg-13-8-99-nb-classifier.qza
scikit_opgg_gg.qza,scikit,open,greengenes,greengenes,scikit_opgg_gg.qza,Y,classify-sklearn_gg-13-8-99-nb-classifier.qza
vsearch_cl_gg_gg.qza,vsearch,closed,greengenes,greengenes,vsearch_cl_gg_gg.qza,Y,refdb_tax_gg.qza


In [4]:
#List of all variables to test
test_list = rownames(benchmarkdf)
test_list = test_list[1]
test_list

[1] "blast_cl_gg_gg.qza"

In [5]:
CreatePhyloObject <- function(tax_in,tab_in){
    #Read OTUS
    otus<-read_qza(tab_in)

    #Read Greengenes taxonomy file
    taxonomy<-read_qza(tax_in)

    #Create taxonomy table
    tax_table = data.frame()
    col = 1
    row = 1

    for (i in taxonomy$data$Taxon){
        tax_list = as.list(strsplit(as.character(i),";"))
        for (i in 1:lengths(tax_list)){
            tax_table[row,col] = tax_list[[1]][i]
            col = col + 1
        }
        col = 1
        row = row + 1
    }
    rownames(tax_table)<-taxonomy$data$Feature.ID
    colnames(tax_table)<-c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
    tax_convert<-as.matrix(tax_table)  

    #read metadata
    metadata<-read.table(manifest_file,sep='\t', header=T, row.names=1, comment="")

    #Create phylo object
    phy_obj<-phyloseq(otu_table(otus$data, taxa_are_rows = T), tax_table(tax_convert), sample_data(metadata))
    print(phy_obj) 
    return(phy_obj)
} 

In [6]:
#Read OTUS
otus<-read_qza(paste(qza_dir,test_list,sep=""))

#Read taxonomy file
taxonomy<-read_qza(paste(ref_dir,benchmarkdf[test_list,"classifier"],sep=""))

#convert tax to matrix for phyloseq input
gen.data <- function(x){
    tax_list = unlist(strsplit(as.character(x),";"))
    return(tax_list)
}
taxonomy_list <- as.list(taxonomy$data$Taxon)
taxonomy_table <- do.call("rbind", lapply(taxonomy_list, gen.data))
colnames(taxonomy_table) <- c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
rownames(taxonomy_table)<-taxonomy$data$Feature.ID
tax_convert<-as.matrix(taxonomy_table)

#read metadata
metadata<-read.table(manifest_file,sep='\t', header=T, row.names=1, comment="")

In [26]:
otus

files.Name,files.Length,files.Date,size
<chr>,<dbl>,<dttm>,<dbl>
c1feadd8-aa89-461a-a26c-50b3cc6d0d40/metadata.yaml,106,2020-05-11 18:38:00,0
c1feadd8-aa89-461a-a26c-50b3cc6d0d40/checksums.md5,8526,2020-05-11 18:38:00,0
c1feadd8-aa89-461a-a26c-50b3cc6d0d40/VERSION,39,2020-05-11 18:38:00,0
c1feadd8-aa89-461a-a26c-50b3cc6d0d40/provenance/citations.bib,4288,2020-05-11 18:38:00,0
c1feadd8-aa89-461a-a26c-50b3cc6d0d40/provenance/metadata.yaml,106,2020-05-11 18:38:00,0
c1feadd8-aa89-461a-a26c-50b3cc6d0d40/provenance/VERSION,39,2020-05-11 18:38:00,0
c1feadd8-aa89-461a-a26c-50b3cc6d0d40/provenance/action/action.yaml,5452,2020-05-11 18:38:00,0
c1feadd8-aa89-461a-a26c-50b3cc6d0d40/provenance/artifacts/a3544b88-67f1-4f86-9c40-ddde25d79d7b/citations.bib,2987,2020-05-11 18:33:00,0
c1feadd8-aa89-461a-a26c-50b3cc6d0d40/provenance/artifacts/a3544b88-67f1-4f86-9c40-ddde25d79d7b/metadata.yaml,107,2020-05-11 18:33:00,0


In [23]:
#Create phylo object
phy_obj<-phyloseq(otu_table(otus$data, taxa_are_rows = T), tax_table(tax_convert), sample_data(metadata))
print(phy_obj) 
 

ERROR: Error in validObject(.Object): invalid class "otu_table" object: 
 Non-numeric matrix provided as OTU table.
Abundance is expected to be numeric.


In [ ]:
for (variable in test_list){
    print("\n****************************************************************************")
    print(paste("Analysis for ",variable))
    
    #Create the phyloseq object
    tax_file = paste(ref_dir,benchmarkdf[variable,"classifier"],sep="")
    table_file = paste(qza_dir,variable,sep="")
    print(tax_file)
    phy_obj = CreatePhyloObject(tax_file,table_file)  

}

In [ ]:
    
    #Create glomed, subsetted obj
    seq_obj = phy_filt(phy_obj)
    remove(phy_obj)
    
    #Create OTUdf
    OTUdf = otu_extract(seq_obj)
    
    #Create TAXdf
    TAXdf = tax_extract(seq_obj)
    
    #Clean df's
    OTUdf_clean = otu_tax_clean(OTUdf,TAXdf)
    
    #Create TP,FP,FN df
    tax_ref = as.character(benchmarkdf[variable,"tax.ref"])
    if(tax_ref=='silva'){
        CONTdf_use = CONTdf_s
        F1df = bench_compare(OTUdf_clean, CONTdf_use)
    } else{
        CONTdf_use = CONTdf_g
        F1df = bench_compare(OTUdf_clean, CONTdf_use)
    }
    
    #Create TP,FP,FN counts    
    if(benchmarkdf[variable,"group.type"]=="dada" | benchmarkdf[variable,"group.type"]=="dn"){
        bench_key = paste(benchmarkdf[variable,"group.type"],benchmarkdf[variable,"tax.type"],
                         benchmarkdf[variable,"tax.ref"],sep="_")
    } else{
        bench_key = paste(benchmarkdf[variable,"group.type"],benchmarkdf[variable,"group.ref"],
                         benchmarkdf[variable,"tax.type"],benchmarkdf[variable,"tax.ref"],sep="_")
    }         
    F1df_counts = bench_counts(OTUdf_clean,F1df,bench_key,variable)
    temp = F1df_counts
    temp$sample_name = rownames(temp)
    mergedf_f1stats = rbind(mergedf_f1stats,temp)
    
    #Create violin plots for each control - counts
    x_list = c("FN","TP","FP")
    y_labs = c("False negative","True positive", "False positive")
    plot_f1_violin(F1df_counts,x_list,y_labs,"Subject.ID",bench_key,"counts")
    
    #Create violin plots for each control - summary
    x_list = c("TAR","TDR","F1")
    y_labs = c("Taxonmic Accuracy Rate","Taxonmic Detection Rate", "F-Measure")
    plot_f1_violin(F1df_counts,x_list,y_labs,"Subject.ID",bench_key,"summary")

    #calculate relative abundance
    RAdf = cal_relabun(OTUdf_clean)
    RAdf$Sample_name = rownames(RAdf)
    
    #Create scatter plots
    #Determine expected to observed scatter plots, normality, spearman coeff 
    title = paste("Scatterplots of controls: ", variable,sep="")
    EOdf_stats = cal_EO(RAdf,CONTdf_use,variable,title)    
    EOdf_stats_merged = rbind(EOdf_stats_merged,EOdf_stats)   
    
    #Create averaged OTU df
    dissdf = create_av_RAdf(RAdf,bench_key,variable)
    if(benchmarkdf[variable,"tax.ref"]=="silva"){
        dissdf_s = bind_rows(dissdf_s,dissdf)
    } else{
        dissdf_g = bind_rows(dissdf_g,dissdf)
    }
    
    #save files
    #write.csv(OTUdf_clean,paste(stat_dir,"OTUcounts_",bench_key,".csv",sep=""))
    #write.csv(F1df,paste(stat_dir,"F1counts_",bench_key,".csv",sep=""))
    write.csv(F1df_counts,paste(stats_dir,"F1_values_binf_",bench_key,".csv",sep=""))

In [7]:


#Create a merged dfs for all variables
mergedf_f1stats = data.frame()
EOdf_stats_merged = data.frame()
dissdf_s = data.frame()
dissdf_g = data.frame()

#List of all variables to test
test_list = unique(benchmarkdf$x)
#test_list = test_list[1]
#test_list

for (variable in test_list){
    print("\n****************************************************************************")
    print(paste("Analysis for ",variable))
    
    #Create the phyloseq object
    tax_file = paste(data_dir,'taxonomic_classification/',variable,sep="")
    table_file = paste(data_dir,'taxonomic_classification/',benchmarkdf[variable,"table.file"],sep="")
    phy_obj = phylo_create(tax_file,table_file)  
    
    #Create glomed, subsetted obj
    seq_obj = phy_filt(phy_obj)
    remove(phy_obj)
    
    #Create OTUdf
    OTUdf = otu_extract(seq_obj)
    
    #Create TAXdf
    TAXdf = tax_extract(seq_obj)
    
    #Clean df's
    OTUdf_clean = otu_tax_clean(OTUdf,TAXdf)
    
    #Create TP,FP,FN df
    tax_ref = as.character(benchmarkdf[variable,"tax.ref"])
    if(tax_ref=='silva'){
        CONTdf_use = CONTdf_s
        F1df = bench_compare(OTUdf_clean, CONTdf_use)
    } else{
        CONTdf_use = CONTdf_g
        F1df = bench_compare(OTUdf_clean, CONTdf_use)
    }
    
    #Create TP,FP,FN counts    
    if(benchmarkdf[variable,"group.type"]=="dada" | benchmarkdf[variable,"group.type"]=="dn"){
        bench_key = paste(benchmarkdf[variable,"group.type"],benchmarkdf[variable,"tax.type"],
                         benchmarkdf[variable,"tax.ref"],sep="_")
    } else{
        bench_key = paste(benchmarkdf[variable,"group.type"],benchmarkdf[variable,"group.ref"],
                         benchmarkdf[variable,"tax.type"],benchmarkdf[variable,"tax.ref"],sep="_")
    }         
    F1df_counts = bench_counts(OTUdf_clean,F1df,bench_key,variable)
    temp = F1df_counts
    temp$sample_name = rownames(temp)
    mergedf_f1stats = rbind(mergedf_f1stats,temp)
    
    #Create violin plots for each control - counts
    x_list = c("FN","TP","FP")
    y_labs = c("False negative","True positive", "False positive")
    plot_f1_violin(F1df_counts,x_list,y_labs,"Subject.ID",bench_key,"counts")
    
    #Create violin plots for each control - summary
    x_list = c("TAR","TDR","F1")
    y_labs = c("Taxonmic Accuracy Rate","Taxonmic Detection Rate", "F-Measure")
    plot_f1_violin(F1df_counts,x_list,y_labs,"Subject.ID",bench_key,"summary")

    #calculate relative abundance
    RAdf = cal_relabun(OTUdf_clean)
    RAdf$Sample_name = rownames(RAdf)
    
    #Create scatter plots
    #Determine expected to observed scatter plots, normality, spearman coeff 
    title = paste("Scatterplots of controls: ", variable,sep="")
    EOdf_stats = cal_EO(RAdf,CONTdf_use,variable,title)    
    EOdf_stats_merged = rbind(EOdf_stats_merged,EOdf_stats)   
    
    #Create averaged OTU df
    dissdf = create_av_RAdf(RAdf,bench_key,variable)
    if(benchmarkdf[variable,"tax.ref"]=="silva"){
        dissdf_s = bind_rows(dissdf_s,dissdf)
    } else{
        dissdf_g = bind_rows(dissdf_g,dissdf)
    }
    
    #save files
    #write.csv(OTUdf_clean,paste(stat_dir,"OTUcounts_",bench_key,".csv",sep=""))
    #write.csv(F1df,paste(stat_dir,"F1counts_",bench_key,".csv",sep=""))
    write.csv(F1df_counts,paste(stats_dir,"F1_values_binf_",bench_key,".csv",sep=""))
}

In [6]:
#Read in taxonomy file
CONTdf = read.csv('../taxonomy/Taxonomy_Complete.csv',stringsAsFactors = FALSE)

#Convert the tax file to SILVA naming with "D_5__"
CONTdf_s = CONTdf
for (i in 1:nrow(CONTdf)){
    old = as.character(CONTdf[i,"Genus"])
    new = paste("D_5__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_s[i,"Genus"] = new
    
    old = as.character(CONTdf[i,"Family"])
    new = paste("D_4__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_s[i,"Family"] = new
}

#Convert the tax file to gg naming with "f"
CONTdf_g = CONTdf
for (i in 1:nrow(CONTdf)){
    old = as.character(CONTdf[i,"Genus"])
    new = paste("g__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_g[i,"Genus"] = new
    
    old = as.character(CONTdf[i,"Family"])
    new = paste("f__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_g[i,"Family"] = new
}

In [8]:
phy_filt <- function(phy_in){
    #glom to tax level, subset for only seq controls
    phy_obj = tax_glom(phy_in,taxrank="Genus") #only merges genera that have the same upper taxonomy
    phy_obj = subset_samples(phy_obj,Sample.Type=="Seq_Control" & Sample.Des!="Seq_Blank")
    return(phy_obj)
}

In [9]:
otu_extract <- function(phy_in){
    # Extract abundance matrix from the phyloseq object
    OTU1 = as(otu_table(phy_in), "matrix")
    if(taxa_are_rows(phy_in)){OTU1 <- t(OTU1)} # transpose
    OTUdf = as.data.frame(OTU1)# Coerce to data.frame
    return(OTUdf)
}

In [10]:
tax_extract <- function(phy_in){
    #extract taxonomy information into dataframe
    TAXdf = tax_table(phy_in)
    return(TAXdf)
}

In [11]:
otu_tax_clean <- function(OTU_in,TAX_in){
    #Remove zero cols
    OTUdf_clean = OTU_in[, which(colSums(OTU_in) != 0)]

    #rename col from features to genus
    for (i in colnames(OTU_in)){
        newname = paste(as.character(TAX_in[i,"Family"]),as.character(TAX_in[i,"Genus"]),sep="")
        
        #standardize gg naming
        newname = trimws(newname) #remove first space in gg naming
        newname = sub(" g__","g__",newname)
        newname = sub("\\[","",newname)
        newname = sub("\\]","",newname)
        
        if(is.na(newname)){
            next;
        } else{
            names(OTUdf_clean)[names(OTUdf_clean) == i] <- newname
        }
    }
    
    #add metadata col
    metadata<-read.table('../manifest/m_cleaned.txt',sep='\t', header=T, row.names=1, comment="")
    for (a in rownames(OTUdf_clean)){
        OTUdf_clean[a,"Subject.ID"] = metadata[a,"Subject.ID"]
    }
    OTUdf_clean = data.frame(OTUdf_clean,stringsAsFactors=FALSE)
    
    return(OTUdf_clean)
}

In [12]:
#Create TP,FP,FN dataframe
bench_compare <- function(OTUdf_in,CONTdf_in){
    
    F1df = OTUdf_in
    F1df$Sample_names = rownames(F1df) #create col of sample_names
    
    #For each of the controls in the cleaned df
    for (cont in unique(OTUdf_in$Subject.ID)){
        temp=filter(CONTdf_in, get(cont) !=0) #create df of taxa related to control
        
        #Create taxa list, merging family and genus - this will help avoid id errors from taxonomic differences
        #IE one taxa labeled at the genus level may be from family1 and the same taxa may also be from family2
        taxa_list=list()
        for (x in rownames(temp)){taxa_list = c(taxa_list,paste(temp[x,"Family"],temp[x,"Genus"],sep=""))}  
        
        #Create list of samples that match the control
        sample_list = filter(F1df, Subject.ID == cont)$Sample_names
        
        #If the data is not in the df (FN), then add the taxa as a new col
        for (taxa in taxa_list){ 
            if(!(taxa %in% unique(colnames(F1df)))){
                F1df[taxa[!(taxa %in% colnames(F1df))]] = 0
            } 
        }

        for (sample in sample_list){
            for (col in unique(colnames(F1df))[unique(colnames(F1df)) != "Subject.ID" & unique(colnames(F1df)) != "Sample_names"]){ 
                #removes subjectID, sample_names cols from loop
                value_check = F1df[sample,col]

                if(col %in% taxa_list && value_check > 0){ #column matches expected taxa, has features
                    F1df[sample,col] = "TP"
                } else if (col %in% taxa_list && value_check == 0){ #column matches expected taxa, no features
                    F1df[sample,col] = "FN"
                } else if ((!col %in% taxa_list) && value_check > 0){ #column doesnt match expected taxa, has features
                    F1df[sample,col] = "FP"
                } else{ #column doesnt match expected taxa, has no features
                    next
                }
            }
        }
    }
    return(F1df)
}

In [13]:
#Create counts of FP, FN, TP
bench_counts <- function(OTUdf_in,F1_in,bmark_in,var_in){
    F1df_counts = subset(OTUdf_in, select=c("Subject.ID"))

    for (sample in rownames(F1df)){
        FN = rowSums(F1_in=="FN")[sample][[1]]
        FP = rowSums(F1_in=="FP")[sample][[1]]
        TP = rowSums(F1_in=="TP")[sample][[1]]

        F1df_counts[sample,"check"]= FN + TP
        F1df_counts[sample,"FN"] = FN
        F1df_counts[sample,"FP"] = FP
        F1df_counts[sample,"TP"] = TP
        F1df_counts[sample,"TAR"] = TP / (TP + FP)
        F1df_counts[sample,"TDR"] = TP / (TP + FN)

        num = F1df_counts[sample,"TAR"]*F1df_counts[sample,"TDR"]
        den = F1df_counts[sample,"TAR"]+F1df_counts[sample,"TDR"] 
        F1df_counts[sample,"F1"] = 2*(num/den)
        F1df_counts[sample,"Subject.ID"] = OTUdf_in[sample,"Subject.ID"]
        
        F1df_counts[sample,"bmark"] = bmark_in
        F1df_counts[sample,"group.type"] = benchmarkdf[variable,"group.type"]  
        F1df_counts[sample,"group.ref"] = benchmarkdf[variable,"group.ref"]
        F1df_counts[sample,"tax.type"] = benchmarkdf[variable,"tax.type"]
        F1df_counts[sample,"tax.ref"] = benchmarkdf[variable,"tax.ref"]
    }
    return(F1df_counts)
}

In [14]:
#F1_in=F1df_counts
bench_means <- function(F1_in){
    df = with(F1_in, tapply(X = FN,       
                    INDEX = Subject.ID,   
                    FUN = mean,     
                    na.rm = TRUE))
    return(df)
}

In [15]:
plot_violin <- function(df_in,x_lab,y_ax,y_lab,col_in){
    p = ggplot(df_in, aes(x=get(x_lab), y=get(y_ax), color=get(col_in))) + 
        geom_violin(trim=FALSE) + 
        labs(color = col_in) +
        theme(axis.text.x = element_text(angle = 90, hjust = 1),
              axis.title.x = element_blank(), 
             legend.position = "none") + 
        ylab(y_lab) +
        stat_summary(fun.data=mean_sdl, color="red")
    return (p)
}

In [16]:
plot_f1_violin <- function(F1_in,x_list,y_list,id_in,title,ftype){
    
    p_1 = plot_violin(F1_in,id_in,x_list[1],y_list[1],id_in)
    p_2 = plot_violin(F1_in,id_in,x_list[2],y_list[2],id_in)
    p_3 = plot_violin(F1_in,id_in,x_list[3],y_list[3],id_in)

    #Create count violin plots
    fig = ggarrange(p_1, p_2, p_3, 
              labels = c("A", "B", "C"),
              title = title,
              #common.legend = TRUE,
              ncol = 2, nrow = 2)
    annotate_figure(fig, #https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
                   top = text_grob(title, color = "blue", face = "bold", size = 14)
    )
    
    ggsave(paste(img_dir,"violin_f1_binf_",ftype,"_",title,".jpg",sep=""))
}

In [17]:
cal_relabun <- function(df_in){
    temp = df_in
    temp$Total = rowSums(df_in[,c(-(ncol(df_in)))])
    col_list = subset(colnames(temp),colnames(temp)!="Subject.ID" & colnames(temp)!="Total")

    #cal_relabund <- function(x){}
    for (row in rownames(temp)){
        for (col in col_list){
            val = temp[row,col]
            if(val==0){
                next
            } else{
                temp[row,col] = val/temp[row,"Total"]
            }
        }
    }
    return(temp)
}

<h2> Taxa Database <h2>

In [18]:
create_taxadf <- function(cont,contdf_in){
    taxdf=filter(contdf_in, get(cont) !=0) #create df of taxa related to control
    taxdf$taxa.id=paste(taxdf$Family,taxdf$Genus,sep="") #create fam/genus col
    taxdf = taxdf[,c("taxa.id",cont),drop=FALSE] #remove all cols but the merged and control col
    taxdf = aggregate(get(cont) ~ taxa.id, data = taxdf, sum) #collapse df for any dups    
    names(taxdf)[2] = cont
    rownames(taxdf) = taxdf$taxa.id
    return(taxdf)
}

<h2> Expected to Observed Correlation <h2>

In [19]:
cal_EO <- function(df_in,contdf_in,bench_in,title_in){
    count=1
    plot_list = list()
    EOdf_merged = data.frame()
    eo_out = data.frame()
    
    #For each of the controls in the cleaned df
    for (cont in unique(df_in$Subject.ID)){

        taxdf = create_taxadf(cont,contdf_in)
        taxa_list=taxdf$taxa.id
        
        #Create list of samples that match the control
        sample_list = filter(RAdf, Subject.ID == cont)$Sample_name
        EOdf= data.frame()
        
        for (taxa in taxa_list){
            for(sample in sample_list){
                EOdf[nrow(EOdf)+1,"Sample_name"] = sample
                EOdf[nrow(EOdf),"taxa.id"] = taxa
                EOdf[nrow(EOdf),"Subject.ID"] = cont
                EOdf[nrow(EOdf),"exp"] = taxdf[taxa,cont]
                EOdf[nrow(EOdf),"bench"] = bench_in

                if(is.null(df_in[sample,taxa])){
                    next
                } else{
                    EOdf[nrow(EOdf),"obs"] = df_in[sample,taxa]
                }
            }
        }
        EOdf_merged = rbind(EOdf_merged,EOdf) #Merge dfs
        
        #Create scatter plots
        plot_list[[count]] = plot_EO(EOdf,cont)
        count=count+1
        
        #Calc stats
        EOdf[is.na(EOdf)] = 0
        print(EOdf)
        temp = cal_EO_stattest(EOdf,cont,bench_in)
        eo_out = rbind(eo_out,temp) #merge df       
    }
    
    #print plots
    fig = ggarrange(plot_list[[1]], plot_list[[2]], plot_list[[3]], plot_list[[4]], plot_list[[5]], plot_list[[6]], 
              labels = c("A", "B", "C", "D", "E", "F"),
              #top = as.character(title_in),
              #common.legend = TRUE#,
              ncol = 2, nrow = 3)
    annotate_figure(fig, top = text_grob(title_in, color = "blue", face = "bold", size = 14)
    )
    ggsave(paste(img_dir,"scatter_binf_",bench_in,".jpg",sep=""))

    return(eo_out)
}

In [20]:
plot_EO <- function(df_in,cont_in){
    my_data=subset(df_in,Subject.ID==cont_in)

    p = ggscatter(my_data, x = "obs", y = "exp", 
              add = "reg.line", conf.int = TRUE, 
              cor.coef = TRUE, cor.method = "pearson",
              xlab = "Expected Rel Abunance (%)", ylab = "Observed Rel Abun (%)") +
        ggtitle(paste("Scatter plot of Rel Abun:",cont_in,sep=""))
    return(p)
}

In [21]:
cal_EO_stattest <- function(df_in,cont_in,bench_in){
    my_data=subset(df_in,Subject.ID==cont_in)
    df_out=data.frame()
    
    df_out[nrow(df_out)+1,"bmark"] = bench_in
    df_out[nrow(df_out),"cont"] = cont_in
    df_out[nrow(df_out),"shapiro"] = shapiro.test(my_data$obs)$p.value
    df_out[nrow(df_out),"spear.p"] = cor.test(my_data$obs,my_data$exp,method = "spearman")$p.value
    df_out[nrow(df_out),"spear.rho"] = cor.test(my_data$obs,my_data$exp,method = "spearman")$estimate[[1]]

    #-1 indicates a strong negative correlation : this means that every time x increases, y decreases (left panel figure)
    #0 means that there is no association between the two variables (x and y) (middle panel figure)
    #1 indicates a strong positive correlation : this means that y increases with x (right panel figure)
    return(df_out)
}

<h2>Dissimilarity <h2>

In [22]:
#Create averaged relative abundance df
create_av_RAdf <- function(RAdf_in,bench_in,var_in){
    tmp = aggregate(RAdf_in[,1:(ncol(RAdf_in)-3)], list(RAdf_in$Subject.ID), mean)
    tmp$bmark=bench_in
    tmp$db = benchmarkdf[var_in,"tax.ref"]
    return(tmp)
}

In [23]:
#Add expected values to RA average dfs
RAdf_addcontrols<-function(cont_in,df_in){
    
    tmp=get(df_in)
    cont_leng = length((unique(OTUdf_clean$Subject.ID)))
    
    for(i in 1:cont_leng){
        cont = as.character(tmp[i,"Group.1"])
        taxadf = create_taxadf(cont,get(cont_in))
        tmp_count = nrow(tmp)+1
        
        for(tx_row in 1:nrow(taxadf)){
            tmp[tmp_count,"Group.1"]=cont
            tmp[tmp_count,taxadf[tx_row,"taxa.id"]] = taxadf[tx_row,cont]
            tmp[tmp_count,"bmark"]="expected"
            tmp[tmp_count,"db"]=tmp[i,"db"]
        }
        
    }
    tmp[is.na(tmp)] = 0 
    return(tmp)
}

In [24]:
calc_dist <- function(RA_in,CONTdf_in){
    rownames(RA_in)=paste(RA_in$Group.1,RA_in$bmark,sep="-")
    RA_in = subset(RA_in, select = -c(db,Group.1,bmark))

    taxa_list<-unique(paste(CONTdf_in$Family,CONTdf_in$Genus,sep=""))
    df_clean = RA_in[,colnames(RA_in) %in% taxa_list] #remove cols not expected

    df_out = vegdist(df_clean,method="bray")   
    return(df_out)
}

In [25]:
plot_dist_cluster <- function(df_in,ref_in){
    fname = paste(img_dir,"hclust_summary_bc_binf_allgroups_",ref_in,".png",sep="")
    png(fname,width = 900, height = 600)
    p = plot(hclust(df_in), cex = .9, main = "All Groups - Bray-Curtis Dissimilarity",
             xlab="Bioinformatic Variables", sub="")
    dev.off()
}

In [26]:
plot_dist_violin <- function(df_in,ref_in){
    df_in = as.data.frame(as.matrix(df_in))
    
    df_plot=data.frame()
    for (row in rownames(df_in)){
        splits = strsplit(row,"-")
        if(splits[[1]][2]=="expected"){
            next
        }else{
            df_plot[nrow(df_plot)+1,"dist"]=df_in[row,paste(splits[[1]][1],"expected",sep="-")]
            df_plot[nrow(df_plot),"name"]=splits[[1]][2]
        }
    }

    p = ggplot(df_plot, aes(x=name, y=dist, color=name)) + 
            geom_violin(trim=FALSE) + 
            labs(color = df_plot$name) +
            theme(axis.text.x = element_text(angle = 90, hjust = 1),
                  axis.title.x = element_blank(), 
                 legend.position = "none") + 
            xlab("Benchmark ID") +
            ylab("Bray-Curtis Dissimilarity") +
            stat_summary(fun.data=mean_sdl, color="red")
    
    
    fig = ggarrange(p, 
              #labels = c("A"),
              #top = as.character(title_in),
              #common.legend = TRUE#,
              ncol = 1, nrow = 1)
    annotate_figure(fig, top = text_grob("Bray-Curtis Dissimilarity", color = "blue", face = "bold", size = 14)
    )
    ggsave(paste(img_dir,"violin_bc_summary_binf_allgroups_",ref_in,".jpg",sep=""))
    return(df_plot)

}

<h2> Statistics <h2>

In [27]:
tstat_paired_f1 <- function(f1_in,subgroup_in){
    
    df_out = data.frame()
    
    cal_list = c("TAR","TDR","F1")
    
    for (cal in cal_list){
    
        comb_list = combn(unique(f1_in[,subgroup_in]),2)
        count=1
        
        for (i in 1:(length(comb_list)/2)){
            
            tstatdf=data.frame()
            var1 = as.character(comb_list[count])
            var2 = as.character(comb_list[count+1])
            count=count+2

            df = f1_in %>%
                filter(f1_in[,subgroup_in] == var1 | f1_in[,subgroup_in] == var2) %>%
                select(subgroup_in, cal)
            
            tmp = t.test(get(cal) ~ get(subgroup_in), data = df)
            tstatdf[nrow(tstatdf)+1,"level"] = cal
            tstatdf[nrow(tstatdf),"comparison"] = paste(var1,var2,sep="_")
            tstatdf[nrow(tstatdf),"pval"] = tmp$p.value
            if(tmp$p.value<0.05){
                tstatdf[nrow(tstatdf),"sig"] = "sig"
            } else{
                tstatdf[nrow(tstatdf),"sig"] = "ns"
            }
            

            df_out = bind_rows(df_out,tstatdf)
        }
    }
        
    return(df_out)
}

In [28]:
adonis_bc <- function(df_in){
    
    for(i in 1:nrow(df_in)){
        split = strsplit(df_in[i,"name"],"_")
        df_in[i,"tax.ref"]=split[[1]][length(split[[1]])] #add col for ref db
        
        if(length(split[[1]])==4){ #dada or dn do not have add' tax ref so they are then 2, otherwise 3
            df_in[i,"tax.type"]=split[[1]][3]
        } else{
            df_in[i,"tax.type"]=split[[1]][2]
        }
        
        df_in[i,"group.type"]=split[[1]][1]  
    }

    cal_list = c("tax.ref","tax.type","group.type")
    df_out=data.frame()
    
    for (cal in cal_list){
        st = adonis(df_in[,1] ~ df_in[,cal],
                    data=df_in, permutations = 99, method="bray")
        df_out[nrow(df_out)+1,"level"] = cal
        df_out[nrow(df_out),"comparison"] = "bray-curtis"
        df_out[nrow(df_out),"pval"] = as.numeric(st$aov.tab$"Pr(>F)"[1])
        
        if(df_out[nrow(df_out),"pval"]<0.05){
            df_out[nrow(df_out),"sig"] = "sig"
        } else{
            df_out[nrow(df_out),"sig"] = "ns"
        }   
    }
    
    dist_s = vegdist(subset(dissdf_s,select = -c(bmark,db,Group.1)),na.rm=TRUE)
    dist_g = vegdist(subset(dissdf_g,select = -c(bmark,db,Group.1)),na.rm=TRUE)
    
    df_out[nrow(df_out)+1,"homo_s"] = anova(betadisper(dist_s,dissdf_s$bmark))$"Pr(>F)"[1]
    df_out[nrow(df_out)+1,"homo_g"] = anova(betadisper(dist_g,dissdf_g$bmark))$"Pr(>F)"[1]
    
    return(df_out)
}

<h2> Run Production Code <h2>

In [32]:
write.csv(EOdf_stats_merged,paste(stats_dir,"EO_stats_binf_summary.csv",sep=""))
write.csv(mergedf_f1stats,paste(stats_dir,"F1_values_summary_binf.csv",sep=""))

<h2> Benchmarked Assessment<h2>

In [33]:
Distdf_merged=data.frame()
#Silva
if(length(test_list)>1){
    RAdf_oe = RAdf_addcontrols("CONTdf_s","dissdf_s")
    RAdf_clean = calc_dist(RAdf_oe,CONTdf_s)
    plot_dist_cluster(RAdf_clean,"s")
    Distdf = plot_dist_violin(RAdf_clean,"s")
} 
Distdf_merged = bind_rows(Distdf_merged,Distdf)

#GreenGenes
RAdf_oe = RAdf_addcontrols("CONTdf_g","dissdf_g")
RAdf_clean = calc_dist(RAdf_oe,CONTdf_g)
plot_dist_cluster(RAdf_clean,"g")
Distdf = plot_dist_violin(RAdf_clean,"g")
Distdf_merged = bind_rows(Distdf_merged,Distdf)

Saving 6.67 x 6.67 in image



png 
  2

Saving 6.67 x 6.67 in image



In [34]:
adonis_stats = adonis_bc(Distdf_merged)
adonis_stats

,level,comparison,pval,sig,homo_s,homo_g
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
1,tax.ref,bray-curtis,0.01,sig,NA,NA
2,tax.type,bray-curtis,0.55,ns,NA,NA
3,group.type,bray-curtis,1.00,ns,NA,NA
4,NA,NA,NA,NA,1,NA
5,NA,NA,NA,NA,NA,0.9999574


In [35]:
#Overall violin plots
x_list = c("TAR","TDR","F1")
y_labs = c("Taxonmic Accuracy Rate","Taxonmic Detection Rate", "F-Measure")

#Create violin plots collapsed to the group.type (merges all ref dbs, and tax class methods)
plot_f1_violin(mergedf_f1stats,x_list,y_labs,"group.type","Group Types","summary")

#Create violin plots collapsed to the tax.type (merges all ref dbs, and grouping methods)
plot_f1_violin(mergedf_f1stats,x_list,y_labs,"tax.type","Tax Types","summary")

#Create violin plots collapsed to the tax.ref (merges all tax class methods and grouping methods)
plot_f1_violin(mergedf_f1stats,x_list,y_labs,"tax.ref","Tax Ref Types","summary")

#Create violin plots for all binf variations (merges all variables)
plot_f1_violin(mergedf_f1stats,x_list,y_labs,"bmark","All Grouping and Assignment Variables","summary")

Warning message in as_grob.default(plot):
"Cannot convert object of class character into a grob."
Saving 6.67 x 6.67 in image

Warning message in as_grob.default(plot):
"Cannot convert object of class character into a grob."
Saving 6.67 x 6.67 in image

Warning message in as_grob.default(plot):
"Cannot convert object of class character into a grob."
Saving 6.67 x 6.67 in image

Warning message in as_grob.default(plot):
"Cannot convert object of class character into a grob."
Saving 6.67 x 6.67 in image



In [36]:
#T-tests
tstatdf_merged=data.frame()
variable_list = c("group.type","tax.type","tax.ref")

for (variable in variable_list){
    tstatdf = tstat_paired_f1(mergedf_f1stats,variable)
    tstatdf_merged=bind_rows(tstatdf_merged,tstatdf)  
}
write.csv(tstatdf_merged,paste(stats_dir,"ttest_stats_binf.csv",sep=""))

Note: Using an external vector in selections is ambiguous.
i Use `all_of(subgroup_in)` instead of `subgroup_in` to silence this message.
i See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.

Note: Using an external vector in selections is ambiguous.
i Use `all_of(cal)` instead of `cal` to silence this message.
i See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.

